In [ ]:
from src import *
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
video_path = "data/Validation_Videos/RGB/DRONE_083193.mp4"
checkpoint_path = "checkpoints/rgb_model_1.pth"
IMAGE_DIR = 'data/track_video/split_A/IR/test/train_images'
LABEL_DIR = 'data/track_video/split_A/IR/test/train_labels'
IMAGE_SIZE = (320, 256)
WINDOW_SIZE = 30
device = 'cuda:1'

In [ ]:
video = utils.load_video_to_numpy(video_path,
                                  target_size=(640, 512)
                                  )
video=torch.from_numpy(video).float()
video=[video[i] for i in range(video.shape[0]) if i % 2 == 0]
video = torch.stack(video, dim=0)
video = video.to(device)

In [ ]:
model = SmallDetectorModule(
    gray_img=True,
    window_size=WINDOW_SIZE,
    
)
checkpoint = torch.load(checkpoint_path, map_location=device)
model.SegmentationModel.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval() 

In [ ]:
video.shape

In [ ]:
heatmaps, bboxes = model(video)

In [ ]:
video.shape

In [ ]:
utils.save_video_with_bboxes(
    video_tensor=video,
    bboxes_per_frame=bboxes,
    output_path="outputs/DRONE_083193_2.mp4",
    fps=15
)

In [ ]:
gbsub_video = model.BackgroundSubtractor(video)

In [ ]:
utils.save_video_from_numpy(gbsub_video, 'outputs/bgsub_video.mp4')

In [ ]:
bboxes[0]

In [ ]:
utils.save_video_from_numpy(heatmaps, 'outputs/fullvideo-c.mp4')